In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score

In [2]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"  
username = "neo4j"     
password = "foodContamination" #This is whatever password you set when you are created your database    

driver = GraphDatabase.driver(uri, auth=(username, password))

In [3]:
cypher_query = """MATCH (n) 
OPTIONAL MATCH (n)-[r]->(m) 
RETURN n, r, m"""

In [4]:
with driver.session() as session:
    result = session.run(cypher_query)

    # Extract properties and relationships from nodes
    data = []
    for record in result:
        node = record['n']
        properties = dict(node.items())
        relationship = record['r']
        if relationship is not None:
            relationship_properties = dict(relationship.items())
        else:
            relationship_properties = {}
        data.append({**properties, **relationship_properties})

    # Create DataFrame from the data
    df = pd.DataFrame(data)

In [5]:
df2 = df[['commodity','eventID', 'gtin', 'sgln', 'cteDate', 'pgln', 'contaminated', 'shortDescription']]
#df2 = df[[ 'commodity', 'shortDescription','cteDate', 'gtin', 'contaminated']]
df2 = df2.dropna(how='all')

In [6]:
df2['cteDate'] = pd.to_datetime(df2['cteDate'])
reference_date = pd.to_datetime('1970-01-01')
df2['cteDate'] = (df2['cteDate'] - reference_date).dt.days.astype(float)

df2['commodity'] = df2['commodity'].fillna(df2['shortDescription'])

In [7]:
df2 = df2.drop('shortDescription', axis=1)

In [8]:
df2['eventID'] = pd.to_numeric(df2['eventID'].str.replace('.', ''))
df2['gtin'] = pd.to_numeric(df2['gtin'].str.replace('.', ''))
df2['sgln'] = pd.to_numeric(df2['sgln'].str.replace('.', ''))
df2['pgln'] = pd.to_numeric(df2['pgln'].str.replace('.', ''))
#df2['parentID'] = pd.to_numeric(df2['parentID'].str.replace('.', ''))

C:\Users\andre\AppData\Local\Temp\ipykernel_22488\1544355506.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['eventID'] = pd.to_numeric(df2['eventID'].str.replace('.', ''))
C:\Users\andre\AppData\Local\Temp\ipykernel_22488\1544355506.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['gtin'] = pd.to_numeric(df2['gtin'].str.replace('.', ''))
C:\Users\andre\AppData\Local\Temp\ipykernel_22488\1544355506.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['sgln'] = pd.to_numeric(df2['sgln'].str.replace('.', 

In [9]:
# Define the columns to be one-hot encoded
columns_to_encode = ['commodity']

# Create the ColumnTransformer
transformer = ColumnTransformer([('one_hot_encoder', OneHotEncoder(), columns_to_encode)], remainder='passthrough')

# Apply the transformation
df_encoded = transformer.fit_transform(df2)

# Convert the encoded data to a DataFrame
df3 = pd.DataFrame(df_encoded.toarray(), columns=transformer.get_feature_names_out(df2.columns))

In [10]:
#columns_to_drop = ['contaminated']
#X = df_encoded.loc[:, ~df_encoded.columns.isin(columns_to_drop)]
X = df3.drop('remainder__contaminated', axis=1)
y = df3['remainder__contaminated']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
model = LogisticRegression()
model.fit(X_train, y_train)

C:\Users\andre\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [13]:
# Predict probabilities for the positive class
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Compute AUC
auc = roc_auc_score(y_test, y_pred_proba)
print("AUC:", auc)

AUC: 0.5


In [14]:
# Get the feature names
feature_names = X_train.columns

# Get the coefficients from the model
coefficients = model.coef_[0]

# Create a DataFrame to display the feature names and coefficients
feature_importance = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

# Sort the DataFrame by coefficient magnitude (absolute value)
feature_importance = feature_importance.reindex(feature_importance['Coefficient'].abs().sort_values(ascending=False).index)

# Print the feature importance
print(feature_importance)

                                        Feature  Coefficient
0          one_hot_encoder__commodity_Egg Salad          0.0
94         one_hot_encoder__commodity_jack fish          0.0
96         one_hot_encoder__commodity_jackfruit          0.0
97   one_hot_encoder__commodity_jalapeno pepper          0.0
98              one_hot_encoder__commodity_kale          0.0
..                                          ...          ...
50         one_hot_encoder__commodity_baby leaf          0.0
51         one_hot_encoder__commodity_barracuda          0.0
52   one_hot_encoder__commodity_barracuda filet          0.0
53             one_hot_encoder__commodity_basil          0.0
148                             remainder__pgln          0.0

[149 rows x 2 columns]
